# Redes Neuronales
## Fashion detector 

In [ ]:
# dependencias necesarias
from pathlib import Path

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import seaborn as sns

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Dropout, Convolution2D, MaxPooling2D, Flatten
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.datasets import fashion_mnist

from sklearn.metrics import accuracy_score, confusion_matrix

# configuración para que las imágenes se vean dentro del notebook
%matplotlib inline

In [ ]:
# obtenemos las imágenes (x) y salidas o categorías (y) del dataset
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

### 1) Análisis exploratorio sobre el conjunto de datos.  
Este es un dataset de 70000 imágenes en blanco y negro de 28x28 pixeles, de 10 categorías de prendas, divididas en un set de train con 60000 imágenes y otro de test de 10000. 

In [ ]:
print('Cantidad y tamaño de imágenes de train:')
x_train.shape  

In [ ]:
print('Cantidad y tamaño de imágenes de test:')
x_test.shape

In [ ]:
# clases de prendas
CLASES = ["T-shirt/top", "Trouser", "Pullover", 
          "Dress", "Coat", "Sandal", "Shirt",
          "Sneaker", "Bag", "Ankle boot"]

Ejemplos de la imágenes sin modificaciones:

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(x_train[i], cmap=plt.cm.binary)
    plt.xlabel(CLASES[y_train[i]])
plt.show()

Las categorías de prendas son las siguientes:

In [ ]:
for clase in CLASES:
    print(clase)

A partir de los gráficos siguientes, podemos notar que el set de datos está completamente balanceado tanto en train como test. En el set de train, hay 6000 imágenes de cada tipo de prenda, mientras que en el set de test hay 1000 imágenes de cada uno.

In [ ]:
# función para contar y graficar la cantidad de prendas por tipo
def distribucion(salidas, titulo=''):
    CANTIDADES = [0,0,0,0,0,0,0,0,0,0]
    for salida in salidas:
        CANTIDADES[salida] += 1
        
    display(titulo)
    plt.pie(CANTIDADES, labels=CLASES, autopct="%0.1f %%")

In [ ]:
distribucion(y_train, 'Distribución de train')

In [ ]:
distribucion(y_test, 'Distribución de test')

### 2) Machine Learning. 

##### Reescalar imágenes  
En primera instancia reescalamos los valores de las imágenes, tanto en test como en train. Esto se puede comprobar en los siguientes gráficos que muestran el rango de valores que posee una imagen del dataset antes y después de reescalar.

In [ ]:
# reescalamos los valores de las imágenes
x_train_r = x_train/ 255.0
x_test_r = x_test / 255.0

In [ ]:
print('Antes de escalar:')
plt.figure()
plt.imshow(x_train[0])
plt.colorbar()
plt.grid(False)
plt.show()

In [ ]:
print('Después de escalar:')
plt.figure()
plt.imshow(x_train_r[0])
plt.colorbar()
plt.grid(False)
plt.show()

##### Modificar el tamaño de las imágenes 
En cuanto al tamaño de las imágenes, optamos por NO modificarlo, debido a que ya es lo suficientemente pequeño como para entrenar sin demorar demasiado.

In [ ]:
# función para graficar la curva de aprendizaje
def curva_aprendizaje(historial):
    plt.plot(historial.history['accuracy'], label='train')
    plt.plot(historial.history['val_accuracy'], label='test')
    plt.title('Accuracy over train epochs')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(loc='upper left')
    plt.show()

In [ ]:
# función para graficar la matriz de confusión
def matriz_confusion(modelo, dt_x, dt_y, title=''):
    predictions = modelo.predict(dt_x)
    pred_label = [np.argmax(i) for i in predictions]
    labels = dt_y
    
    conf_matrix = confusion_matrix(labels, pred_label)

    ax = sns.heatmap(conf_matrix, 
                cmap='Blues', 
                xticklabels=CLASES, 
                yticklabels=CLASES,
                annot=True,
                fmt='d')

    plt.xlabel('Predicted class') 
    plt.ylabel('True class') 
    
    print(title)
    plt.show()

#### RED NEURONAL 1:
* Tipo: MLP.
* Capas: 4 densas, con 30, 20, 20 y 10 neuronas en ese orden.
* Dropout: 20% en cada capa.
* Función de activación: 'tanh' en cada capa y 'softmax' en la salida.
* Épocas: 25.
* Tamaño del batch: 250.

In [ ]:
model_mlp_1 = Sequential([
    
    Flatten(input_shape=(28, 28)),
    
    Dense(30, activation='tanh'),
    Dropout(0.2),
    
    Dense(20, activation='tanh'),
    Dropout(0.2),
            
    Dense(20, activation='tanh'),
    Dropout(0.2),

    Dense(len(CLASES), activation='softmax'),
])

model_mlp_1.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy',],
)
    
model_mlp_1.summary()

In [ ]:
history_mlp_1 = model_mlp_1.fit(
    x_train_r,
    y_train,
    epochs=25,
    batch_size=250,
    validation_data=(x_test_r, y_test)
)

In [ ]:
curva_aprendizaje(history_mlp_1)

In [ ]:
matriz_confusion(model_mlp_1, x_train_r, y_train, 'Matriz de confusión - Train')

In [ ]:
matriz_confusion(model_mlp_1, x_test_r, y_test, 'Matriz de confusión - Test')

#### RED NEURONAL 2:
* Tipo: MLP.
* Capas: 4 densas, con 30, 20, 20 y 10 neuronas en ese orden.
* Dropout: 50% en cada capa.
* Función de activación: 'tanh' en cada capa y 'softmax' en la salida.
* Épocas: 25.
* Tamaño del batch: 250.

In [ ]:
model_mlp_2 = Sequential([
    
    Flatten(input_shape=(28, 28)),
    
    Dense(30, activation='tanh'),
    Dropout(0.5),
    
    Dense(20, activation='tanh'),
    Dropout(0.5),
            
    Dense(20, activation='tanh'),
    Dropout(0.5),

    Dense(len(CLASES), activation='softmax'),
])

model_mlp_2.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy',],
)
    
model_mlp_2.summary()

In [ ]:
history_mlp_2 = model_mlp_2.fit(
    x_train_r,
    y_train,
    epochs=25,
    batch_size=250,
    validation_data=(x_test_r, y_test)
)

In [ ]:
curva_aprendizaje(history_mlp_2)

In [ ]:
matriz_confusion(model_mlp_2, x_train_r, y_train, 'Matriz de confusión - Train')

In [ ]:
matriz_confusion(model_mlp_2, x_test_r, y_test, 'Matriz de confusión - Test')

#### RED NEURONAL 3:
* Tipo: MLP.
* Capas: 10 densas, con 30, 30, 30, 20, 20, 20, 10, 10, 10 y 10 neuronas en ese orden.
* Dropout: 20% en cada capa.
* Función de activación: 'tanh' en cada capa y 'softmax' en la salida.
* Épocas: 25.
* Tamaño del batch: 250.

In [ ]:
model_mlp_3 = Sequential([
    
    Flatten(input_shape=(28, 28)),
    
    Dense(30, activation='tanh'),
    Dropout(0.2),
    
    Dense(30, activation='tanh'),
    Dropout(0.2),
    
    Dense(30, activation='tanh'),
    Dropout(0.2),
    
    Dense(20, activation='tanh'),
    Dropout(0.2),
            
    Dense(20, activation='tanh'),
    Dropout(0.2),
    
    Dense(20, activation='tanh'),
    Dropout(0.2),
    
    Dense(10, activation='tanh'),
    Dropout(0.2),
    
    Dense(10, activation='tanh'),
    Dropout(0.2),
    
    Dense(10, activation='tanh'),
    Dropout(0.2),

    Dense(len(CLASES), activation='softmax'),
])

model_mlp_3.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy',],
)
    
model_mlp_3.summary()

In [ ]:
history_mlp_3 = model_mlp_3.fit(
    x_train_r,
    y_train,
    epochs=25,
    batch_size=250,
    validation_data=(x_test_r, y_test)
)

In [ ]:
curva_aprendizaje(history_mlp_3)

In [ ]:
matriz_confusion(model_mlp_3, x_train_r, y_train, 'Matriz de confusión - Train')

In [ ]:
matriz_confusion(model_mlp_3, x_test_r, y_test, 'Matriz de confusión - Test')

#### RED NEURONAL 4:
* Tipo: MLP.
* Capas: 4 densas, con 30, 20, 20 y 10 neuronas en ese orden.
* Dropout: 20% en cada capa.
* Función de activación: 'relu' en cada capa y 'softmax' en la salida.
* Épocas: 25.
* Tamaño del batch: 250.

In [ ]:
model_mlp_4 = Sequential([
    
    Flatten(input_shape=(28, 28)),
    
    Dense(30, activation='relu'),
    Dropout(0.2),
    
    Dense(20, activation='relu'),
    Dropout(0.2),
            
    Dense(20, activation='relu'),
    Dropout(0.2),

    Dense(len(CLASES), activation='softmax'),
])

model_mlp_4.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy',],
)
    
model_mlp_4.summary()

In [ ]:
history_mlp_4 = model_mlp_4.fit(
    x_train_r,
    y_train,
    epochs=25,
    batch_size=250,
    validation_data=(x_test_r, y_test)
)

In [ ]:
curva_aprendizaje(history_mlp_4)

In [ ]:
matriz_confusion(model_mlp_4, x_train_r, y_train, 'Matriz de confusión - Train')

In [ ]:
matriz_confusion(model_mlp_4, x_test_r, y_test, 'Matriz de confusión - Test')

#### RED NEURONAL 5:
* Tipo: MLP.
* Capas: 4 densas, con 80, 80, 60 y 10 neuronas en ese orden.
* Dropout: 20% en cada capa.
* Función de activación: 'tanh' en cada capa y 'softmax' en la salida.
* Épocas: 25.
* Tamaño del batch: 250.

In [ ]:
model_mlp_5 = Sequential([
    
    Flatten(input_shape=(28, 28)),
    
    Dense(80, activation='tanh'),
    Dropout(0.2),
    
    Dense(80, activation='tanh'),
    Dropout(0.2),
            
    Dense(60, activation='tanh'),
    Dropout(0.2),

    Dense(len(CLASES), activation='softmax'),
])

model_mlp_5.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy',],
)
    
model_mlp_5.summary()

In [ ]:
history_mlp_5 = model_mlp_5.fit(
    x_train_r,
    y_train,
    epochs=25,
    batch_size=250,
    validation_data=(x_test_r, y_test)
)

In [ ]:
curva_aprendizaje(history_mlp_5)

In [ ]:
matriz_confusion(model_mlp_5, x_train_r, y_train, 'Matriz de confusión - Train')

In [ ]:
matriz_confusion(model_mlp_5, x_test_r, y_test, 'Matriz de confusión - Test')